# Agilepy science API tutorial

## Importing the library

In [ ]:
from agilepy.api.AGAnalysis import AGAnalysis

In [ ]:
from os.path import expandvars
from pathlib import Path
from shutil import rmtree

In [ ]:
%matplotlib notebook

## Output directory 

In [ ]:
outputFolder = Path(expandvars("$HOME/agilepy_science_api_tutorial/output"))

if Path(outputFolder).exists():
    print("Deleting",outputFolder)
    rmtree(outputFolder)

## Configuration file

In [ ]:
config = """
input:
  evtfile: $AGILE/agilepy-test-data/evt_index/agile_proc3_fm3.119_asdc2_EVT.index
  logfile: $AGILE/agilepy-test-data/log_index/agile_proc3_data_asdc2_LOG.log.index

output:
  outdir: %s
  filenameprefix: science_api_tutorial
  logfilenameprefix: science_api_tutorial_log
  verboselvl: 1

selection:
  tmin: 456361778
  tmax: 456537945
  timetype: TT
  glon: 79.8
  glat: 0.7
 
maps:
  energybins:
    - 100, 300
    - 300, 1000
  fovbinnumber: 2

model:
  galcoeff: 0.6, 0.8
  isocoeff: 10, 15
  
"""%(outputFolder)

with open("./agilepy_conf.yaml", "w") as cf:
    cf.write(config)

## Initialization of the class

In [ ]:
ag = AGAnalysis("./agilepy_conf.yaml")

## API: configuration file 

In [ ]:
ag.printOptions()

In [ ]:
ag.printOptions("maps")

In [ ]:
ag.getOption("energybins")

In [ ]:
ag.setOptions(energybins=[[100,300], [300, 1000]], mapsize=50, binsize=0.4)

In [ ]:
ag.printOptions("maps")

## API: sources

In [ ]:
cat2agl = ag.getSupportedCatalogs().pop()
cat2agl

In [ ]:
!head -n 5 "$AGILE/catalogs/2AGL.multi"

In [ ]:
sources = ag.loadSources(cat2agl, rangeDist = (0, 20))

In [ ]:
print(sources[0])

In [ ]:
type(sources[0])

### Selecting the sources
The selection criteria can be expressed using the following Source class’s parameters:

* name: the unique code identifying the source.
* dist: the distance of the source from the center of the maps.
* flux: the flux value.
* sqrtTS: the radix square of the ts.

In [ ]:
selectedSources = ag.selectSources('flux > 0')

In [ ]:
len(selectedSources)

In [ ]:
selectedSources = ag.selectSources('name == "2AGLJ1048-5836"')

In [ ]:
len(selectedSources)

In [ ]:
selectedSources = ag.selectSources("flux > 0 AND dist <= 4", quiet=True)

In [ ]:
len(selectedSources)

### Free/Fix source parameters
You can fix or free the following parametes:

* flux
* index
* index1
* index2
* cutoffEnergy
* pivotEnergy
* curvature
* index2

In [ ]:
sources = ag.freeSources('name == "2AGLJ1048-5836"', "flux", True)

In [ ]:
for s in sources:
    print(s)

In [ ]:
sources = ag.freeSources('name == "2AGLJ1048-5836"', "index", True)

In [ ]:
for s in sources:
    print(s)

In [ ]:
sources = ag.freeSources("flux > 0 AND dist <= 4", "flux", True)

In [ ]:
len(sources)

In [ ]:
for s in sources:
    print(s)

### Deleting sources

In [ ]:
deleted = ag.deleteSources('name == "2AGLJ2029+4403"').pop()
print(deleted)

In [ ]:
len(ag.getSources())

### Adding a source

In [ ]:
newSourceDict = {
    "glon" : 81.9171,
    "glat": 3.0854,
    "spectrumType" : "PowerLaw",
    "flux": 9.97261e-08,
    "index": 1.92501
}

if ag.addSource("2AGLJ2029+4403", newSourceDict):
    print("Source loaded")

In [ ]:
len(ag.getSources())

In [ ]:
if ag.addSource("2AGLJ2029+4403", deleted):
    print("Source loaded")
else:
    print("Source is already present in the SourcesLibrary")

## API: science

### Generating maps

In [ ]:
ag.printOptions("maps")

In [ ]:
maplistfile = ag.generateMaps()

### Displaying maps

In [ ]:
ag.displayCtsSkyMaps(smooth=True, sigma=3)

In [ ]:
ag.displayExpSkyMaps()

In [ ]:
ag.displayGasSkyMaps()

### Saving maps

In [ ]:
ag.displayCtsSkyMaps(smooth=True, sigma=3, saveImage=True)

### Estimation of background coefficients

In [ ]:
print("isocoeff: {} galcoeff: {}".format(ag.getOption("isocoeff"), ag.getOption("galcoeff")))

In [ ]:
# This method will require data coming from 14 days before tmin. Hence, you can't execute this method if you are using: 

# evtfile: $AGILE/agilepy-test-data/evt_index/agile_proc3_fm3.119_asdc2_EVT.index
# logfile: $AGILE/agilepy-test-data/log_index/agile_proc3_data_asdc2_LOG.log.index

# ag.calcBkg("2AGLJ2202+4214")

In [ ]:
print("isocoeff: {} galcoeff: {}".format(ag.getOption("isocoeff"), ag.getOption("galcoeff")))

### Maximum likelyhood analysis

In [ ]:
sourceFiles = ag.mle()

In [ ]:
sources = ag.selectSources("sqrtTS > 0")